In [1]:
#Importing packages
#
# You can delete whatever package(s) you want..
#
import os
import json
import time
import requests
import math as m
import pandas as pd
import numpy as np
import koios_python as kp



# Because most of my data sets I have made can have up to 30+ columns and 20+ rows
# code below will increase pandas defaults for max rows 
# and columns that you can display in a Juptyer Notebook
pd.options.display.max_columns = 60
pd.options.display.max_rows = 1000

## NoteBook 1


In this Notebook we will be going over how to use the koios_python library in various scenarios a developer may face. The first thing we need to do is

create a class instance for koios_python and choose what network we would like to use for our queries, in this tutorial we will use mainnet but the 

library supports all the various developer networks available including preview, preprod, guild, and mainnet.

In [2]:
# create a koios server object with network set to mainnet
koios_mainnet = kp.URLs(network='mainnet')

# let's check the current tip of the mainnet
koios_mainnet.get_tip()

[{'hash': '573995bfa97a7ba712d620f49e5b5db28ec2a2e437bcdb1f888f788c22cb3b50',
  'epoch_no': 384,
  'abs_slot': 80634516,
  'epoch_slot': 109716,
  'block_no': 8195970,
  'block_time': 1672200807}]

### Let's go over some use case examples for koios_python

**Native Assets:**

What if you wanted to find out how many Native Assets and tokens are on the cardano blockchain? (too expensive need async XD)

In [4]:
# my own version of the get_asset_list function
def get_asset_list(content_range="0-999"):
    url = "https://api.koios.rest/api/v0/asset_list"
    headers = {'Range': content_range}
    response = requests.get(url, headers=headers)
    try: 
        asset_list = response.json()
        pd_asset_list = pd.DataFrame(asset_list)
        return pd_asset_list
    except Exception as e:
            print(e)

In [7]:
asset_df = pd.DataFrame()
for i in range(2):
        # data = koios_mainnet.get_asset_list(content_range="{}-{}".format(i*1000,(i+1)*1000))
        # print(data)
        # print(koios_mainnet.get_asset_list(content_range="{}-{}".format(i*1000,(i+1)*1000)))
        asset_data = get_asset_list(content_range="{}-{}".format(i*1000,(i+1)*1000))
        # print(asset_data)
        if len(asset_data) > 0:
                asset_df = pd.concat([asset_df, asset_data], axis=0, join='outer')


In [8]:
asset_df

,policy_id,asset_name,fingerprint
0,00000002df633853f6a47465c9496721d2d5b1291b8398...,6e7574636f696e,asset12h3p5l3nd5y26lr22am7y7ga3vxghkhf57zkhd
1,000000adf8fcbdf03a5c154123aff674edf287fb13532a...,5853534e4654,asset1ke42k9ug86yr535qwkq5edlvfdvlpfuslrmfup
2,000006126365b110b339bd39c5b5c0fc89eb146cc186ba...,5061696e7465724e656b6f4f7263686172643031,asset1xedcm62nhn8hv0644apeydjppudtuqxm0nyvxq
3,000006126365b110b339bd39c5b5c0fc89eb146cc186ba...,5061696e7465724e656b6f4f7263686172643032,asset1c8lfga60kvftykq9a65naml07633zqwe883gaj
4,000006126365b110b339bd39c5b5c0fc89eb146cc186ba...,5061696e7465724e656b6f4f7263686172643033,asset1kuezrt73hk9pp93skjl2jezn3etnyhmr6ezmru
...,...,...,...
995,0029cb7c88c7567b63d1a512c0ed626aa169688ec98073...,6c70207a01,asset105qdq03hefge9eez9nvq7dnuwclrf07aqqvpn3
996,0029cb7c88c7567b63d1a512c0ed626aa169688ec98073...,6c70207a02,asset1jenwc47qvwndzc22420n9dytfjd70jm32c6afj
997,0029cb7c88c7567b63d1a512c0ed626aa169688ec98073...,6c70207a03,asset1rm23kft2nx94wvppcfmydcrp3rfkpysvc5uvtp
998,0029cb7c88c7567b63d1a512c0ed626aa169688ec98073...,6c70207b,asset13h488j83sg7sjjcjuhjpr8uydmk7xejcuejwhg


In [9]:
asset_df.drop_duplicates(subset=['fingerprint'], inplace=True)

In [11]:
# want the index to be just the number of rows in the dataframe
asset_df.index = range(len(asset_df))
asset_df

,policy_id,asset_name,fingerprint
0,00000002df633853f6a47465c9496721d2d5b1291b8398...,6e7574636f696e,asset12h3p5l3nd5y26lr22am7y7ga3vxghkhf57zkhd
1,000000adf8fcbdf03a5c154123aff674edf287fb13532a...,5853534e4654,asset1ke42k9ug86yr535qwkq5edlvfdvlpfuslrmfup
2,000006126365b110b339bd39c5b5c0fc89eb146cc186ba...,5061696e7465724e656b6f4f7263686172643031,asset1xedcm62nhn8hv0644apeydjppudtuqxm0nyvxq
3,000006126365b110b339bd39c5b5c0fc89eb146cc186ba...,5061696e7465724e656b6f4f7263686172643032,asset1c8lfga60kvftykq9a65naml07633zqwe883gaj
4,000006126365b110b339bd39c5b5c0fc89eb146cc186ba...,5061696e7465724e656b6f4f7263686172643033,asset1kuezrt73hk9pp93skjl2jezn3etnyhmr6ezmru
...,...,...,...
1995,0029cb7c88c7567b63d1a512c0ed626aa169688ec98073...,6c70207a01,asset105qdq03hefge9eez9nvq7dnuwclrf07aqqvpn3
1996,0029cb7c88c7567b63d1a512c0ed626aa169688ec98073...,6c70207a02,asset1jenwc47qvwndzc22420n9dytfjd70jm32c6afj
1997,0029cb7c88c7567b63d1a512c0ed626aa169688ec98073...,6c70207a03,asset1rm23kft2nx94wvppcfmydcrp3rfkpysvc5uvtp
1998,0029cb7c88c7567b63d1a512c0ed626aa169688ec98073...,6c70207b,asset13h488j83sg7sjjcjuhjpr8uydmk7xejcuejwhg


How many wallets hold a specific asset or token?

In [ ]:
asset_df[0][93]

{'policy_id': '000077e4f562322fc25f79335f7b309a89347626e4d7a64598b0b927',
 'asset_name': '53756d6d65725363656e654461794f6e65303233',
 'fingerprint': 'asset18mlzwmdw2ggyj80ecg7thnjnucppuy70y7ddav'}

In [ ]:
asset_df[0][95]

{'policy_id': '000077e4f562322fc25f79335f7b309a89347626e4d7a64598b0b927',
 'asset_name': '53756d6d65725363656e654461794f6e65303235',
 'fingerprint': 'asset1m3ke2eqguyqfud3gk4pu3l0m7e8078m43evewj'}

Pool information

Pool's history?

How many pools are there?

In [ ]:
asset_df[0][100]

{'policy_id': '000077e4f562322fc25f79335f7b309a89347626e4d7a64598b0b927',
 'asset_name': '53756d6d65725363656e654461794f6e65303330',
 'fingerprint': 'asset1tmusxscqcpycx8ms5ywwhd9vclfqzxykgpzxml'}